In [16]:
from datetime import datetime
import pandas as pd
import yfinance as yf
from sector_mapper import update_sectors
import numpy as np 

In [17]:
end_date = datetime.today().strftime('%Y-%m-%d')
spy = yf.download('SPY', start='1993-01-29', end=end_date)
cpi = pd.read_csv('input_data\cpi.csv')
spy_holdings = pd.read_excel('input_data\holdings-daily-us-en-spy.xlsx')

<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:4: SyntaxWarning: invalid escape sequence '\h'
<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:4: SyntaxWarning: invalid escape sequence '\h'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_161056\1281653963.py:3: SyntaxWarning: invalid escape sequence '\c'
  cpi = pd.read_csv('input_data\cpi.csv')
C:\Users\Administrator\AppData\Local\Temp\ipykernel_161056\1281653963.py:4: SyntaxWarning: invalid escape sequence '\h'
  spy_holdings = pd.read_excel('input_data\holdings-daily-us-en-spy.xlsx')
[*********************100%***********************]  1 of 1 completed
C:\Users\Administrator\AppData\Local\Temp\ipykernel_161056\1281653963.py:3: SyntaxWarning: invalid escape sequence '\c'
  cpi = pd.read_csv('input_data\cpi.csv')
C:\Users\Administrator\AppData\Local\Temp\ipykernel_161056\1281653963.py:4: SyntaxWarning: invalid escape sequence '\h'
  spy_holdings = pd.read_excel('input_data\holdings-daily-us-en-spy.xlsx')


FileNotFoundError: [Errno 2] No such file or directory: 'input_data\\cpi.csv'

In [ ]:
spy_holdings = update_sectors(spy_holdings)
spy_holdings.drop(columns=['Local Currency','Identifier','SEDOL'], inplace= True)

In [ ]:
spy_holdings

,Name,Ticker,Weight,Sector,Shares Held
0,APPLE INC,AAPL,6.757246,Information Technology,1.909512e+08
1,NVIDIA CORP,NVDA,6.044540,Information Technology,3.098777e+08
2,MICROSOFT CORP,MSFT,6.003886,Information Technology,9.392184e+07
3,AMAZON.COM INC,AMZN,3.868495,Consumer Discretionary,1.182215e+08
4,META PLATFORMS INC CLASS A,META,2.742725,Communication Services,2.753946e+07
...,...,...,...,...,...
500,CELANESE CORP,CE,0.012395,Unknown,1.383364e+06
501,CAESARS ENTERTAINMENT INC,CZR,0.012068,Unknown,2.685071e+06
502,FMC CORP,FMC,0.010776,Unknown,1.581247e+06
503,NEWS CORP CLASS B,NWS,0.007222,Unknown,1.421638e+06


In [ ]:
print(spy.columns)
spy = spy.droplevel('Ticker', axis=1)

MultiIndex([( 'Close', 'SPY'),
            (  'High', 'SPY'),
            (   'Low', 'SPY'),
            (  'Open', 'SPY'),
            ('Volume', 'SPY')],
           names=['Price', 'Ticker'])


In [ ]:
cpi = cpi.rename(columns={"DATE": "date", "CPIAUCSL": "cpi_value"})
cpi["date"] = pd.to_datetime(cpi["date"])
cpi["cpi_value"] = cpi["cpi_value"].astype(float).round(2)
cpi.reset_index(inplace=True, drop=True)
cpi = cpi[(cpi["date"] >= "2000-01-01")]

spy.reset_index(inplace=True)
spy = spy.rename(columns={"Close": "close_price", "Date": "date"})
spy["date"] = pd.to_datetime(spy["date"])
spy["close_price"] = spy["close_price"].round(2)
spy.drop(columns=["High","Low","Open","Volume"], inplace=True)

In [ ]:
spy

Price,date,close_price
0,1993-01-29,24.45
1,1993-02-01,24.63
2,1993-02-02,24.68
3,1993-02-03,24.94
4,1993-02-04,25.04
...,...,...
8092,2025-03-24,574.08
8093,2025-03-25,575.46
8094,2025-03-26,568.59
8095,2025-03-27,567.08


In [ ]:
cpi

,date,cpi_value
636,2000-01-01,169.30
637,2000-02-01,170.00
638,2000-03-01,171.00
639,2000-04-01,170.90
640,2000-05-01,171.20
...,...,...
929,2024-06-01,313.05
930,2024-07-01,313.53
931,2024-08-01,314.12
932,2024-09-01,314.69


# Inflation-Adjusting SPY Prices

This code performs several key steps to create inflation-adjusted (real) prices for the S&P 500 ETF (SPY):

1. **Creating an adjustment factor**: 
   ```python
   cpi["adj_fac"] = cpi["cpi_value"] / cpi["cpi_value"].iloc[0]
   ```
   This calculates an inflation adjustment factor by dividing each CPI value by the first CPI value in the dataset, creating a ratio that represents how prices have changed relative to the base period.

2. **Creating a complete date range**:
   ```python
   date_range = pd.date_range(start=spy["date"].min(), end=spy["date"].max(), freq="D")
   full_date_df = pd.DataFrame({"date": date_range})
   ```
   This builds a continuous daily date range spanning the entire SPY dataset period.

3. **Merging SPY data with complete date range**:
   ```python
   spy_full = full_date_df.merge(spy, on="date", how="left")
   ```
   This ensures all calendar days are included, even non-trading days.

4. **Merging with CPI (inflation) data**:
   ```python
   merged_df = spy_full.merge(cpi, on="date", how="left")
   ```
   This combines the SPY price data with the CPI data.

5. **Handling missing values**:
   ```python
   merged_df["close"] = merged_df["close"].ffill()
   merged_df["adj_fac"] = merged_df["adj_fac"].ffill()
   ```
   Forward-filling missing values ensures continuous data for non-trading days and potentially monthly CPI values.

6. **Calculating real (inflation-adjusted) prices**:
   ```python
   merged_df["real_price"] = merged_df["close"] / merged_df["adj_fac"]
   ```
   This adjusts the nominal SPY prices for inflation by dividing by the adjustment factor.

7. **Cleaning the dataset**:
   ```python
   merged_df = merged_df.dropna(subset=["close", "adj_fac"])
   merged_df = merged_df.reset_index(drop=True)
   merged_df = merged_df.dropna(subset=["cpi_value"])
   ```
   This removes any rows with missing critical values and resets the index for consistency.

In [ ]:
cpi["adj_fac"] = cpi["cpi_value"] / cpi["cpi_value"].iloc[0]

date_range = pd.date_range(start=spy["date"].min(), end=spy["date"].max(), freq="D")
full_date_df = pd.DataFrame({"date": date_range})

# Merge SPY with the full date range to include all days
spy_full = full_date_df.merge(spy, on="date", how="left")

# Merge SPY data with CPI data
merged_df = spy_full.merge(cpi, on="date", how="left")

# Forward-fill missing values in both CPI and SPY data
merged_df["close_price"] = merged_df["close_price"].ffill()  # SPY closing prices
merged_df["adj_fac"] = merged_df["adj_fac"].ffill()  # Inflation adjustment factor

# Calculate real_price (inflation-adjusted SPY price)
merged_df["real_price"] = merged_df["close_price"] / merged_df["adj_fac"]

# Drop rows with any remaining missing critical values
merged_df = merged_df.dropna(subset=["close_price", "adj_fac"])

# Reset index for consistency
merged_df = merged_df.reset_index(drop=True)
merged_df = merged_df.dropna(subset=["cpi_value"])

# Display the first few rows of the merged DataFrame
print(merged_df.head())

          date  close_price  cpi_value   adj_fac  real_price
0   2000-01-01        93.33      169.3  1.000000   93.330000
31  2000-02-01        89.56      170.0  1.004135   89.191224
60  2000-03-01        87.97      171.0  1.010041   87.095444
91  2000-04-01        95.80      170.9  1.009451   94.903101
121 2000-05-01        93.69      171.2  1.011223   92.650216


In [ ]:
initial_investment = 10000

# Calculate nominal and real investment values
nominal_inv_10k = (merged_df["close_price"] / merged_df["close_price"].iloc[0]) * initial_investment
real_inv_10k = (merged_df["real_price"] / merged_df["real_price"].iloc[0]) * initial_investment

# Add calculated values to the merged DataFrame
merged_df["nominal_inv_10k"] = nominal_inv_10k
merged_df["real_inv_10k"] = real_inv_10k

print(merged_df.head(5))

          date  close_price  cpi_value   adj_fac  real_price  nominal_inv_10k  \
0   2000-01-01        93.33      169.3  1.000000   93.330000     10000.000000   
31  2000-02-01        89.56      170.0  1.004135   89.191224      9596.057002   
60  2000-03-01        87.97      171.0  1.010041   87.095444      9425.693775   
91  2000-04-01        95.80      170.9  1.009451   94.903101     10264.652309   
121 2000-05-01        93.69      171.2  1.011223   92.650216     10038.572806   

     real_inv_10k  
0    10000.000000  
31    9556.543826  
60    9331.988047  
91   10168.552580  
121   9927.163412  


In [ ]:
sector_allocation = {
    'Information Technology': 30.39,
    'Financials': 14.22,
    'Health Care': 11.20,
    'Consumer Discretionary': 10.11,
    'Communication Services': 9.45,
    'Industrials': 8.45,
    'Consumer Staples': 5.97,
    'Energy': 3.42,
    'Utilities': 2.52,
    'Real Estate': 2.23,
    'Materials': 2.05
}

spy_sector = pd.DataFrame(list(sector_allocation.items()), columns=['Sector', 'Percentage'])
print(spy_sector.head(5))

                   Sector  Percentage
0  Information Technology       30.39
1              Financials       14.22
2             Health Care       11.20
3  Consumer Discretionary       10.11
4  Communication Services        9.45


In [ ]:
spy_sector.to_csv('spy_sectors.csv')
merged_df.to_csv('spy_inflation.csv', index=False)
spy_holdings.to_csv('spy_share_allocation.csv', index=False)

In [ ]:
cpi["cpi_value"] = pd.to_numeric(cpi["cpi_value"], errors="coerce")
cpi = cpi.dropna(subset=["cpi_value"])
cpi["date"] = pd.to_datetime(cpi["date"])
cpi = cpi.set_index("date").resample("MS").mean().reset_index()

cpi["infl_rate"] = ((cpi["cpi_value"] / cpi["cpi_value"].iloc[0]) - 1) * 100

initial_investment = 10000
annual_interest_rate = 0.015
monthly_interest_rate = annual_interest_rate / 12
number_of_months = len(cpi)

nominal_future_value = (
    initial_investment * (1 + monthly_interest_rate) ** number_of_months
)

cumulative_inflation_rate = (cpi["cpi_value"].iloc[-1] / cpi["cpi_value"].iloc[0] - 1) * 100
cumulative_inflation_factor = 1 + (cumulative_inflation_rate / 100)

real_value = nominal_future_value / cumulative_inflation_factor

nominal_values = initial_investment * (1 + monthly_interest_rate) ** np.arange(len(cpi))
real_values = nominal_values / ((1 + (cpi["infl_rate"] / 100)).values)

print(f"Nominal Future Value: ${nominal_future_value:,.2f}")
print(f"Inflation-Adjusted (Real) Value: ${real_value:,.2f}")

Nominal Future Value: $14,510.21
Inflation-Adjusted (Real) Value: $7,787.54


In [ ]:
result_df = pd.DataFrame({
    'date': cpi['date'],
    'nominal_value': nominal_values,
    'real_value': real_values,
    'cpi_value': cpi['cpi_value'],
    'inflation_rate': cpi['infl_rate']
})

import os 
os.chdir('output_data')
result_df.to_csv('investment_values.csv', index=False)

print("DataFrame created and exported to 'investment_values.csv'")

DataFrame created and exported to 'investment_values.csv'
